# 🧠 Building simplified models of primary visual cortex (V1) 🐭 🐵

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/drive/1wZjPChsFO4dyLHIfH95d_IMeMwgsa3xK?usp=sharing)

**By Fengtong(Farah) Du and Carsen Stringer**

In this tutorial, we will build predictive models of V1 neural activity from visual stimuli (natural images). The predictive models are:

1.   linear regression
2.   one convolutional layer (shared across neurons) + neuron-specific readout
3.   2-convolutional layer net + neuron-specific readout with separable convolutions
4.   small neuron-specific "minimodels" with 2 convolutional layers + readout.

For each of these models, we will visualize the weights to help us understand the feature selectivity of V1 neurons.

This notebook uses [code](https://github.com/mouseland/minimodel) and [data](https://janelia.figshare.com/articles/dataset/Towards_a_simplified_model_of_primary_visual_cortex/28797638) from [Du et al 2025](https://www.nature.com/articles/s41467-025-61171-9). If you use any code or data from this notebook, please cite the paper,

> **Total duration: 6–8 hours**

To keep the notebook interactive, there are three types of exercises throughout

1) QUESTION MARKS: where ????? need to be replaced by a short equation, such as a variable or a function name.

2) DISCUSSION QUESTIONS (Q): have a short discussion with your colleague about this. At the end of each section, we will open the discussions to the whole group.

3) QUIZ: multiple-choice that we take across the entire group. Keep track of your own points.

---


# 0. Setup ⏱️ ~5 min  

We will install the minimodel code, download the data, and define some helper functions for plotting and filters.

In google colab, you will want to choose a runtime with a GPU (e.g. T4 option).

## 0.1 Install dependencies

If you are running locally, you will want to remove the "--no-deps" flag.

In [ ]:
# 🛠️ Install minimodel
!pip install --no-deps git+https://github.com/mouseland/minimodel.git
!pip install rastermap
!pip install opt-einsum

## 0.2 Download data

In [ ]:
# Create a data directory
!mkdir -p data
!mkdir -p checkpoints/fullmodel/

# Download files
# neural activity data
!wget --header="User-Agent: Mozilla/5.0" --header="Accept: text/html" --header="Connection: keep-alive" --content-disposition -P data "https://figshare.com/ndownloader/files/53712311"
# natural images that were shown
!wget --header="User-Agent: Mozilla/5.0" --header="Accept: text/html" --header="Connection: keep-alive" --content-disposition -P data "https://janelia.figshare.com/ndownloader/files/53678783"
# checkpoint for full model
!wget -O checkpoints/fullmodel/FX10_051623_2layer_16_320_clamp_norm_depthsep_pool.pt https://github.com/MouseLand/minimodel/releases/download/V1.0.0/FX10_051623_2layer_16_320_clamp_norm_depthsep_pool.pt


## 0.3 Import libraries, declare GPU device

In [ ]:
# Imports
import os
import numpy as np
import time
import torch
from torch import nn
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib import patches

# DECLARE DEVICE
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Using torch device: {device}')

## 0.4 Define helper functions

In [ ]:
# @title run this cell to define the helper functions
# remove spines in rcparams
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

def show_stimulus(img, ax=None, show=False, filters=False, vmax=1):
  """ Visualize a stimulus """
  if ax is None:
    ax = plt.gca()
  if filters:
    ax.imshow(img, vmin=-vmax, vmax=vmax, cmap='RdBu_r')
  else:
    ax.imshow(img+0.5, cmap="gray")
  ax.set_xticks([])
  ax.set_yticks([])
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_visible(False)
  if show:
    plt.show()


def plot_repeats(spks_test, ineuron):
    """ plot repeats of stimulus presentations for ineuron
    Args:
        spks_test: neural activity, shape (nstim x nreps x n_neurons)
        ineuron: index of neuron to plot
    """
    nreps = spks_test.shape[1]
    fig, ax = plt.subplots(nreps + 1, 1, figsize=(10, 5))
    for i in range(nreps):
        ax[i].plot(spks_test[:, i, ineuron], label=f'repeat {i+1}')
        if i == 0:
            ax[i].set_title(f'Neuron {ineuron}')
        ax[i].text(-50, 1, f'repeat {i+1}')
        ax[i].axis('off')

    i += 1
    sp_avg = spks_test[:, :, ineuron].mean(axis=1)
    sp_std = spks_test[:, :, ineuron].std(axis=1)
    ax[i].plot(sp_avg, color='k', lw=1)
    ax[i].fill_between(np.arange(len(sp_avg)), sp_avg - sp_std, sp_avg + sp_std, color='k', alpha=0.2)
    ax[i].text(-50, 1, 'average\n +/- std')
    ax[i].spines['left'].set_visible(False)
    ax[i].spines['bottom'].set_visible(False)
    ax[i].set_yticks([])
    ax[i].set_xlabel('stimulus id')



def plot_example_activations(stimuli, act, filters, channels=[0], vmax=10):
  """ plot activations act and corresponding stimulus
    Args:
        stimuli: stimulus input to convolutional layer (n x h x w) or (h x w)
        act: activations of convolutional layer (n_bins x conv_channels x n_bins)
        filters: filters of convolutional layer (n_filters x 1 x K x K)
        channels: which conv channels to plot

  """
  if stimuli.ndim>2:
    n_stimuli = stimuli.shape[0]
  else:
    stimuli = stimuli.unsqueeze(0)
    n_stimuli = 1

  fig, axs = plt.subplots(n_stimuli+1, 1 + len(channels), figsize=(18, 12))

  ax = axs[0, 0]
  ax.axis('off')

  # plot filters
  for i in range(len(filters)):
    ax = axs[0, i+1]
    show_stimulus(filters[i].squeeze(), ax=ax, filters=True)
    ax.set_title(f'filter {i}')
    ax.axis('off')

  # plot stimulus
  for i in range(n_stimuli):
    show_stimulus(stimuli[i].squeeze(), ax=axs[i+1, 0])
    axs[i+1, 0].set_title('stimulus')

    # plot example activations
    for k, (channel, ax) in enumerate(zip(channels, axs[i+1][1:])):
      im = ax.imshow(act[i, channel], vmin=-vmax, vmax=vmax, cmap='RdBu_r')
      ax.set_xlabel('x-pos')
      ax.set_ylabel('y-pos')
      ax.set_title(f'channel {channel}')

    cb_ax = fig.add_axes([1.05, 0.8, 0.01, 0.1])
    plt.colorbar(im, cax=cb_ax)
    cb_ax.set_title('activation\n strength')
  plt.tight_layout()
  plt.show()

def plot_correlation_maps(spks, act, filters, vmax=0.35):
  """ plot correlation maps of neural activity with convolutional layer activations
  Args:
    spks: neural activity, shape (n_stimuli x n_reps x n_neurons)
    act: activations of convolutional layer (n_neurons x conv_channels x h, w)
    channels: which conv channels to plot
  """
  n_stimuli, n_reps, n_neurons = spks.shape
  n_neurons, n_filters, h, w = act.shape

  fig, axs = plt.subplots(n_neurons+1, n_filters, figsize=(12, 6))

  ax = axs[0, 0]
  ax.axis('off')

  # plot filters
  for i in range(len(filters)):
    ax = axs[0, i]
    show_stimulus(filters[i].squeeze(), ax=ax, filters=True)
    ax.set_title(f'filter {i}')
    ax.axis('off')

  # plot stimulus
  for i in range(n_neurons):
    ax.axis('off')

    # plot example activations
    channels = np.arange(n_filters)
    for k, (channel, ax) in enumerate(zip(channels, axs[i+1])):
      # add gaussian blur to the activation map
      from scipy.ndimage import gaussian_filter
      im = ax.imshow(act[i,k], vmin=-vmax, vmax=vmax, cmap='RdBu_r')
      ax.axis('off')
      if k == 0:
        ax.set_title(f'neuron {i}', loc='left')


    cb_ax = fig.add_axes([1.05, 0.8, 0.01, 0.1])
    plt.colorbar(im, cax=cb_ax)
    cb_ax.set_title('correlation')
  plt.tight_layout()
  plt.show()

def filters(out_channels=6, K=7):
  """ make example filters, some center-surround and gabors
  Returns:
      filters: out_channels x K x K
  """
  grid = np.linspace(-K/2, K/2, K).astype(np.float32)
  xx,yy = np.meshgrid(grid, grid, indexing='ij')

  # create center-surround filters
  sigma = 1.1*2
  gaussian = np.exp(-(xx**2 + yy**2)**0.5/(2*sigma**2))
  wide_gaussian = np.exp(-(xx**2 + yy**2)**0.5/(2*(sigma*2)**2))
  center_surround = gaussian - 0.5 * wide_gaussian

  # create gabor filters
  thetas = np.linspace(0, 360, out_channels-2+1)[:-1] * np.pi/180
  gabors = np.zeros((len(thetas), K, K), np.float32)
  lam = 10*2
  phi = np.pi/2
  gaussian = np.exp(-(xx**2 + yy**2)**0.5/(2*(sigma*0.4)**2))
  for i,theta in enumerate(thetas):
    x = xx*np.cos(theta) + yy*np.sin(theta)
    gabors[i] = gaussian * np.cos(2*np.pi*x/lam + phi)

  filters = np.concatenate((center_surround[np.newaxis,:,:],
                            -1*center_surround[np.newaxis,:,:],
                            gabors),
                           axis=0)
  filters /= np.abs(filters).max(axis=(1,2))[:,np.newaxis,np.newaxis]
  filters -= filters.mean(axis=(1,2))[:,np.newaxis,np.newaxis]
  # convert to torch
  filters = torch.from_numpy(filters)
  # add channel axis
  filters = filters.unsqueeze(1)

  return filters


def plot_Wc(Wc_sort, conv_filters=None, vmax_filt=0.5, vmax_Wc=0.5):
    fig, axs = plt.subplots(1 if conv_filters is None else 2, 1, figsize=(14,5))
    ax = axs if conv_filters is None else axs[0]
    ax.imshow(Wc_sort, cmap='RdBu_r', aspect="auto", vmin=-vmax_Wc, vmax=vmax_Wc)
    if conv_filters is not None:
        ax.set_xticks(np.arange(Wc_sort.shape[1]))
        ax.set_xticklabels(np.arange(1, Wc_sort.shape[1] + 1))
    ax.set_xlabel('convolutional channels')
    ax.set_ylabel('neurons')

    if conv_filters is not None:
        ax = axs[1]
        vmax = 5e-1
        K = conv_filters.shape[-1]
        filt_cat = conv_filters.squeeze().transpose(1, 0, 2).reshape(K, -1)
        ax.imshow(filt_cat, cmap='RdBu_r', vmin=-vmax_filt, vmax=vmax_filt)
        for i in range(len(conv_filters)):
            ax.axvline(i * K - 0.5, color='k', lw=1, ls='--')
        ax.set_xticks(np.arange(len(conv_filters)) * K + K//2)
        ax.set_xticklabels(np.arange(1, len(conv_filters) + 1));

def plot_predictions(Y_test, Y_pred_linear, Y_pred_conv1, Y_pred_fullmodel, k=1):
    Yt = Y_test[:,:,k].mean(axis=1)
    titles = ['true', 'linear', 'conv+readout', '16-320+readout']
    fig, ax = plt.subplots(1, 1, figsize=(14, 4))
    ax.plot(Yt, label='true', color='k', lw=2.5)
    ax.plot(Y_pred_linear[:,k], lw=1)
    ax.plot(Y_pred_conv1[:,k], lw=1)
    ax.plot(Y_pred_fullmodel[:,k],  lw=1)
    ax.set_ylabel('response')
    ax.set_xlabel('stimulus')
    plt.legend(titles, frameon=False, loc='upper left')
    plt.show()

    fig, axs = plt.subplots(1, 3, figsize=(14, 4))
    colors = ['tab:blue', 'tab:orange', 'tab:green']
    for i in range(3):
        Yp = Y_pred_linear[:,k] if i==0 else Y_pred_conv1[:,k] if i==1 else Y_pred_fullmodel[:,k]
        axs[i].scatter(Yt, Yp, c=colors[i])
        axs[i].axis('square')
        axs[i].set_ylabel(titles[i+1], color=colors[i])
        axs[i].set_xlabel(titles[0])
        axs[i].plot([0, Yt.max()], [0, Yt.max()], 'k--')
    plt.tight_layout()
    plt.show()

def add_channel_frame(axs, row, col_start, col_end, color, alpha):
    ax = axs[row, col_start]  # Leftmost axis in the row
    # Rectangle coordinates (x, y) and dimensions (width, height)
    rect = patches.Rectangle(
        (-0.025, -0.05), (col_end - col_start + 1)*1.2, 1.1, transform=ax.transAxes,
        color=color, fill=False, linewidth=3, zorder=10, alpha=alpha,
        clip_on=False  # To ensure it draws outside the axes
    )
    ax.add_patch(rect)

def plot_conv2_1x1(conv1_w, conv2_1x1, ichannels=[127,242, 102,68,204,217,293], Wc=None):
    import matplotlib.colors as mcolors
    norm = mcolors.Normalize(vmin=-1, vmax=1)
    # normalize each channel
    conv2_1x1_norm = conv2_1x1 / np.linalg.norm(conv2_1x1, axis=1, keepdims=True)
    cmap = plt.cm.RdBu_r
    nchannels = len(ichannels)
    n_top = 8
    fig, ax = plt.subplots(nchannels, n_top, figsize=(6, int(0.8*nchannels)))

    for i in range(nchannels):
        channel_w = conv2_1x1_norm[ichannels[i]]
        idxes = np.argsort(np.abs(channel_w))[::-1][:n_top] # keep the top 8 channels
        isort = np.argsort(channel_w[idxes])[::-1]
        isort = idxes[isort]
        for j in range(n_top):
            ax[i, j].imshow(conv1_w[isort[j]], cmap='RdBu_r', vmin=-0.2, vmax=0.2)
            ax[i, j].axis('off')

            # Create the frame color
            frame_color = cmap(norm(channel_w[isort[j]]))
            alpha = abs(channel_w[isort[j]])
            # Add a rectangle frame around the image
            rect = plt.Rectangle(
                (-1, -1),  # Starting point
                conv1_w[isort[j]].shape[1]+1,  # Width
                conv1_w[isort[j]].shape[0]+1,  # Height
                linewidth=7, edgecolor=frame_color, facecolor='none'
            )
            ax[i, j].add_patch(rect)
        if Wc is None:
            ax[i, 0].text(-0.15, 0.4, f'channel {ichannels[i]+1}', transform=ax[i, 0].transAxes, ha="right")
        else:
            ax[i, 0].text(-0.15, 0.4, '\n'.join([f'channel {ichannels[i]+1}',f'w_c = {Wc[i]:.3f}']), transform=ax[i, 0].transAxes, ha="right")
    plt.show()

def plot_conv2(conv1_w, conv2_1x1, ichannels=[127,242, 102,68,204,217,293], Wc=None):
    isort_channels = np.argsort(Wc[ichannels])[::-1]  # sort channels by Wc
    ichannels = ichannels[isort_channels]  # sort channels by Wc
    import matplotlib.colors as mcolors
    norm = mcolors.Normalize(vmin=-1, vmax=1)
    # normalize each channel
    conv2_1x1_norm = conv2_1x1 / np.linalg.norm(conv2_1x1, axis=1, keepdims=True)
    cmap = plt.cm.RdBu_r
    nchannels = len(ichannels)

    # visualize conv2 spatial
    conv2_spatial = model.core.features.layer1.ds_conv.spatial_conv.weight.data.cpu().numpy().squeeze()


    n_top = 8
    fig, ax = plt.subplots(nchannels, n_top+1, figsize=(8, int(0.8*nchannels)))

    for i in range(nchannels):
        channel_w = conv2_1x1_norm[ichannels[i]]
        idxes = np.argsort(np.abs(channel_w))[::-1][:n_top] # keep the top 8 channels
        isort = np.argsort(channel_w[idxes])[::-1]
        isort = idxes[isort]
        for j in range(n_top):
            ax[i, j].imshow(conv1_w[isort[j]], cmap='RdBu_r', vmin=-0.2, vmax=0.2)
            # ax[i, j].set_title(f'{isort[j]:.0f}', Wfontsize=8)
            ax[i, j].axis('off')

            # Create the frame color
            # print(norm(channel_w[isort[j]]))
            frame_color = cmap(norm(channel_w[isort[j]]))
            alpha = abs(channel_w[isort[j]])
            # Add a rectangle frame around the image
            rect = plt.Rectangle(
                (-1, -1),  # Starting point
                conv1_w[isort[j]].shape[1]+1,  # Width
                conv1_w[isort[j]].shape[0]+1,  # Height
                linewidth=7, edgecolor=frame_color, facecolor='none'
            )
            ax[i, j].add_patch(rect)
        if Wc is None:
            ax[i, 0].text(-0.15, 0.4, f'channel {ichannels[i]+1}', transform=ax[i, 0].transAxes, ha="right")
        else:
            ax[i, 0].text(-0.15, 0.4, ' '.join([f'channel {ichannels[i]+1}',f'w_c = {Wc[ichannels[i]]:.3f}']), transform=ax[i, 0].transAxes, ha="right")
        ax[i, -1].imshow(conv2_spatial[ichannels[i]], cmap='RdBu_r', vmin=-0.2, vmax=0.2)
        ax[i, -1].axis('off')
    ax[0, -1].set_title('conv2\nspatial', fontsize=10)
    plt.show()

def plot_maxstim(valid_wc, train_img, prediction_isort, channel_activity_isort, neuron_mask):
    cmap = plt.get_cmap('plasma', 9)
    # Parameters for the second plot
    pad = 5
    vmin = 0
    vmax = 255

    isort = np.argsort(valid_wc)[::-1]
    Nchannel = np.min([len(valid_wc), 8])
    Nimg = channel_activity_isort.shape[0]


    # Combined plot layout
    fig = plt.figure(figsize=(Nimg + 14, 8 * 1.1))
    gs = plt.GridSpec(8, Nimg + 2, figure=fig, hspace=0.2, wspace=0.1)

    # Plot one (4x4 grid on the left side, occupying 2 rows per row)
    nshow = 16
    for i in range(nshow):
        row = (i // 2)
        col = (i % 2)
        ax = fig.add_subplot(gs[row, col])
        ax.imshow(train_img[prediction_isort[i]].cpu().numpy().squeeze() * neuron_mask, cmap='gray', vmin=-1, vmax=1)
        ax.axis('off')
        if i == 0:
            # add a rect around the 16 images
            rect = patches.Rectangle(
                (-0.03, -7.38-1.07), 2.275, 9.5, transform=ax.transAxes,
                color='black', fill=False, linewidth=3, zorder=10,
                clip_on=False  # To ensure it draws outside the axes
            )
            ax.add_patch(rect)
            ax.text(-0.2, 1.3, 'neuron maximum stimuli', transform=ax.transAxes,
                    verticalalignment='center', fontsize=24, color='black')

    # Plot two (8xNimg grid on the right side)
    axs = np.empty((8, Nimg), dtype=object)
    for i in range(Nchannel):
        if i < 6:
            ichannel = i
        else:
            ichannel = -(Nchannel - i)
        for j in range(Nimg):
            axs[i, j] = fig.add_subplot(gs[i, j + 2])
            axs[i, j].imshow(train_img[channel_activity_isort[j, isort[ichannel]]].cpu().numpy().squeeze() * neuron_mask, cmap='gray', vmin=-1, vmax=1)
            axs[i, j].axis('off')
        wc_value = valid_wc[isort[ichannel]]
        # Determine the frame color and linewidth based on valid_wc[isort[ichannel]]
        if wc_value > 0:
            color = 'red'
        else:
            color = 'blue'
        add_channel_frame(axs, i, 0, Nimg - 1, color, np.abs(valid_wc[isort[ichannel]]/np.max(np.abs(valid_wc))))

        ax = axs[i, Nimg - 1]  # Rightmost axis in the row
        if ichannel < 0: ichannel = len(valid_wc) + ichannel
        ax.text(1.2, 0.5, f'channel {ichannel+1}', transform=ax.transAxes,
                verticalalignment='center', fontsize=24, color=cmap(i))
        if i == 0:
            ax = axs[i, Nimg - 1]
            ax.text(-5, 1.3, 'channel maximum stimuli', transform=ax.transAxes,
                    verticalalignment='center', fontsize=24, color='black')
    plt.show()

---

# 1. Data introduction ⏱️ ~15 min  

In each recording there are ~5,000 neurons recorded simultaneously in mouse V1 responding to >30,000 natural and texture images.  

In this section, we will load the image stimuli, neural response arrays, and associated metadata:

- **`imgs`**: A collection of 68,000 images, each of shape `(66, 130)`.
  - These include:
    - 59,500 natural images presented at most one time during the recording session, used for training the model.
    - 500 of these natural images presented 10 times during the recording session, used for testing the model.
    - 8,000 texture images, used to assess texture-invariant representations (see more details on this in the paper).

- **Neural recordings**: >30,000 images were shown to the mouse during the experiment, 500 of these images were repeated 10 times to create a test set with trial-averaged responses.
  - **`spks`**: stimuli by neurons matrix
  - **`istim_train`**: indices of the `imgs` matrix corresponding to each stimulus presentation in `spks`
  - **`spks_test`**: stimuli by trials by neurons matrix
  - **`istim_test`**: indices of the `imgs` matrix corresponding to each stimulus in `spks_test`
  - **`ypos`, `xpos`**: vectors of length neurons with the position of each neuron in the recording

Let's start by loading the images and neural responses using the provided utility functions.


In [ ]:
from minimodel import data

mouse_id = 3

data_path = './data'
weight_path = './checkpoints'

# load images
imgs = data.load_images(data_path, mouse_id, file=data.img_file_name[mouse_id])

# load neural activity data
fname = '%s_nat60k_%s.npz'%(data.db[mouse_id]['mname'], data.db[mouse_id]['datexp'])
spks, istim_train, istim_test, xpos, ypos, spks_test = data.load_neurons(file_path = os.path.join(data_path, fname), mouse_id = mouse_id)

ypos, xpos = ypos / 0.5, xpos / 0.75 # convert to um, imaging at 0.5 x 0.75 um / pixel

n_stim_train, n_neurons_all = spks.shape
n_stim_test, n_reps, _ = spks_test.shape
print('Loaded %d neurons w/ %d stimuli presented for training'%(n_neurons_all, n_stim_train))
print('Loaded %d neurons w/ %d stimuli presented %d times for testing'%(n_neurons_all, n_stim_test, n_reps))

## 1.1 Visualize stimulus

First, let's visualize the stimuli:

In [ ]:
# plot example stimuli
h_ = 3
n_col = 8
h, w  = imgs[0].shape  # height and width of stimulus

np.random.seed(1)

################################################################################
# TODO for students: randomly sample example stimulus indices from the test set
raise NotImplementedError("Student exercise: sample test stimuli to visualize")
################################################################################
# Hint:
# - Use np.random.choice to select h_ * n_col examples from istim_test
# - Set replace=False to avoid duplicates
example_stims = ...

example_imgs = imgs[example_stims]

fig, axs = plt.subplots(h_, n_col, figsize=(n_col, h_))
for i in range(n_col * h_):
  ax = axs[i // n_col, i % n_col]
  ax.imshow(example_imgs[i], cmap='gray')
  ax.axis('off')
  ax.set_title(f'stim {example_stims[i]}')
fig.suptitle(f'stimulus size: {h} x {w}')
plt.tight_layout()
plt.show()

## 1.2 Visualize neural responses

Next, we visualize the neural responses. The dataset contains neural recordings from approximately 5,000 neurons in mouse V1. We will sort the neurons by their correlation for visualization and bin across neurons, using our algorithm *rastermap* (this will take ~1 min).


In [ ]:
from rastermap import Rastermap

model = Rastermap(n_PCs=200, bin_size=25).fit(spks.T)

plt.figure(figsize=(10, 5))

################################################################################
# TODO for students: visualize the embedding
raise NotImplementedError("Student exercise: visualize rastermap embedding")
################################################################################
# Hint:
# - model.X_embedding has shape (n_super_neurons, n_stimuli)
# - What do these dimensions represent?
# - Try showing a subset of the embedding (e.g., first 1000 stimuli)
plt.imshow(..., cmap='gray_r', vmin=0, vmax=2, aspect='auto')


plt.title('Rastermap embedding of neural activity')
plt.xlabel('stimulus id')
plt.ylabel('"super" neurons (bins of 25 neurons)')

We can see neurons which have diverse responses per stimulus, and then neurons that seem to not respond to the stimuli. We want to focus on modeling neurons which are driven by stimuli, so we will choose stimulus-responsive neurons in the next section, based on response reliability.

---

# 2. Response Reliability: fraction of explainable variance ⏱️ ~20 min  

For the test set, we presented 500 images, each shown 10 times. These repeated stimulus presentations allow us to measure the trial-to-trial consistency of each neuron.

Here we plot the activity for a single neuron across trials:


In [ ]:
# check activity
print('neural activity (test set):', spks_test.shape)
nstim, nreps, n_neurons = spks_test.shape

ineuron = 50
plot_repeats(spks_test, ineuron)

As in Willeke et al and Cadena et al, we can quantify the fraction of variance a neuron has related to the stimulus - the Fraction of Explainable Variance (FEV) - as

$$FEV = \frac{\text{var}[Y] - \sigma^2_{\text{noise}}}{\text{var}[Y]}.$$

$Y$ is the matrix for a single neuron of stimuli $i$ by repeats $j$, $\text{var}[Y]$ is the total variance across all images and repeats, and the noise across repeats $\sigma^2_{\text{noise}}$ is defined as

$$\sigma^2_{\text{noise}} =\mathbb{E}_i [\text{var}_j[Y]] $$

the average across stimuli of the per stimulus variance across repeats.

For the example neuron above:

In [ ]:
sp = spks_test[:, :, ineuron]


################################################################################
# TODO for students: calculate the total variance and noise variance for a single neuron
raise NotImplementedError("Student exercise: compute total and noise variance for a single neuron, then FEV")
################################################################################
# Hint:
# - total variance: variance across all responses, ignoring trial structure
# - noise variance: average variance across trials (per image)
# - Use ddof=1 for an unbiased estimator of the variance
# - Can you figure out why we use ddof=1 instead of 0?
noise_var = ...
total_var = ...

fev = (total_var - noise_var) / total_var

print(f'fraction of explainable variance: {fev:.3f}')

We can now compute this for each neuron as

In [ ]:
################################################################################
# TODO for students: calculate the total and noise variance for all neurons
raise NotImplementedError("Student exercise: compute FEV for all neurons")
################################################################################
# Hint:
# - spks_test has shape (n_stimuli, n_trials, n_neurons)
# - Compute noise variance by averaging variance across trials for each stimulus
# - Compute total variance across all responses per neuron
# - Use ddof=1 to get an unbiased estimate
noise_var = ...
total_var = ...

FEV_all = (total_var - noise_var) / total_var

print(f'FEV average +/- std: {FEV_all.mean():.3f} +/- {FEV_all.std():.3f}')

fig, ax = plt.subplots(1, 1, figsize=(4, 3))
ax.hist(FEV_all, bins=20, color='lightgray', edgecolor='black')
ax.set_xlabel('FEV')
ax.set_ylabel('# of neurons')
ax.set_title('FEV Distribution')
plt.tight_layout()

**Q: What's the difference between a neuron with high FEV and a neurons with low FEV? And what would influence the FEV of the neuron?**

In Du et al, we focused on neurons with FEV > 0.15. For this tutorial we will increase the threshold to 0.4 to reduce the number of neurons to work with and to just visualize the most reliable neurons.

In [ ]:
igood = np.where(FEV_all > 0.4)[0]
spks = spks[:, igood]
spks_test = spks_test[:, :, igood]
FEV = FEV_all[igood]
ypos = ypos[igood]
xpos = xpos[igood]
n_neurons = len(igood)
print(f'Number of neurons with FEV > 0.4: {n_neurons}')
print(f'average FEV of these neurons: {FEV.mean():.3f}')

---

# 3. Baseline: Linear Regression ⏱️ ~30-40 min  

Before introducing nonlinear models, we establish a performance baseline using a linear regression model to predict neural responses $Y$ directly from image pixels $X$ with a matrix $A$, such that the prediction $\hat{Y} = XA$. We will also visualize the linear receptive fields of each neuron, they are defined by $A$.

## 3.1 Fitting linear regression model

We will fit $A$ such that the mean squared error between the neural responses and the prediction is minimized: $\text{argmin}_A \sum(Y - XA)^2$. The solution which minimizes the cost function is

$$ A = (X_\text{train}^\top X_\text{train})^{-1} (X_\text{train}^\top Y_\text{train})$$

where $X$ is stimuli by pixels and $Y$ is stimuli by neurons.

(optional) can you derive this equation yourself by finding the minimum of the cost function, i.e. taking the derivative of the cost function and setting it to zero and solving for $A$?

To regularize the linear regression (called ridge regression), we can add a $\lambda$ term to the diagonal of the input covariance matrix before inversion:

$$ A = (X_\text{train}^\top X_\text{train} + \lambda I)^{-1} (X_\text{train}^\top Y_\text{train})$$


For the linear regression, we will z-score the neural activity and subtract the mean pixel values across stimuli - this means that $Y$ and $X$ are mean zero and we do not need to predict a bias term. We will also bin the stimuli across pixels to reduce the size of $A$.

In [ ]:
################################################################################
# TODO for students: normalize the data (z-score)

raise NotImplementedError("Student exercise: normalize Y_train and Y_test using z-scoring")
################################################################################
# Hint:
# - Subtract the mean and divide by the std across stimuli for each neuron
# - This ensures all neurons are on the same scale
# Question: Why is z-scoring important for linear models?
Y_train = spks.copy()
Y_train -= ...
Y_train /= ...

Y_test = spks_test.copy()
Y_test -= ...
Y_test /= ...

# downsample the images by binning the stimuli in X and Y
b = 2
X = imgs.copy()
X = np.reshape(X[:, : (h//b) * b, : (w//b) * b], (X.shape[0], h//b, b, w//b, b))
X = X.mean(axis=(2,4))

# flatten pixels and subtract mean pixel values across stimuli
X = X.reshape(X.shape[0], -1)
X -= X.mean(axis=0)
X_train = X[istim_train]
X_test = X[istim_test]

Compute $A$ from $X_{\text{train}}$ and $Y_{\text{train}}$.

(optional) you can try playing with the regularizer - how does this affect performance?

(optional) try using the GPU to speed up the linear regression, with the matrix multiplications on the GPU and by using `torch.linalg.solve`.

In [ ]:
# regularized linear regression from stimuli to neural responses
################################################################################
# TODO for students: calculate the covariance matrices XtX and XtY
raise NotImplementedError("Student exercise: compute XtX and XtY for linear regression")
################################################################################
# Hint:
# - XtX is the stimulus covariance matrix (XᵀX)
# - XtY is the stimulus-response covariance matrix (XᵀY)
# - X_train has shape (n_samples, n_pixels)
# - Y_train has shape (n_samples, n_neurons)
XtX = ...
XtY = ...

lam = 5e4 # regularizer

# add a ridge regularizer to the linear regression with parameter "lam"
XtX = XtX + lam * np.eye(X_train.shape[1])

# regression matrix
A = np.linalg.solve(XtX, XtY)

## 3.2 Linear receptive fields

We can now visualize some of the per-neuron regression matrices, the columns of $A$ - these are the linear receptive fields of the neurons:

In [ ]:
################################################################################
# TODO for students: what are we visualizing here?
raise NotImplementedError("Student exercise: interpret the visualization of RFs")
################################################################################
# Hint:
# - A is the weight matrix from linear regression: (n_pixels × n_neurons)
# - Each column in A represents how one neuron weights each image pixel
# - By reshaping A into 2D maps, we can visualize the spatial receptive field for each neuron
# - These receptive fields show *where* in the image each neuron is most sensitive to
# - What does this tell us about each neuron’s tuning?
RF = ...  # reshape to receptive fields

h_, n_col = 3, 6  # number of rows and columns for plotting
ineurons = np.random.choice(A.shape[1], n_col * h_, replace=False)
fig, axs = plt.subplots(h_, n_col, figsize=(n_col * 2, h_ * 2))
for i in range(n_col * h_):
    ax = axs[i // n_col, i % n_col]
    show_stimulus(RF[ineurons[i]], ax=ax, filters=True, vmax=3*RF.std(axis=(1,2)).mean())
    ax.set_title(f'neuron {ineurons[i]}')
    ax.axis('off')

Each neuron receives input from a particular part of the visual field, as we can see with the receptive fields. The receptive field center for each neuron varies continuously across primary visual cortex (V1). We can visualize it for this recording from a small part of V1, we will compute the RF center by smoothing the absolute value of the RFs. Each pixel in the image is ~1 degree in size, so we will rescale by the binning size to convert to visual angle.

In [ ]:
from scipy.ndimage import gaussian_filter

RF_sm = gaussian_filter(np.abs(RF.copy()), 2, axes=(1, 2))
################################################################################
# TODO for students: find the RF center for each neuron
raise NotImplementedError("Student exercise: compute imax, ymax, xmax for RF center")
################################################################################
# Hint:
# - RF_sm has shape (n_neurons, height, width)
# - First flatten each RF map and find the index of the max value (imax)
# - Then convert imax into 2D coordinates using np.unravel_index
# - RF_sm.shape[1:] gives the spatial (y, x) dimensions
imax = ...
ymax, xmax = ...

# convert to degrees of visual angle
ymax = b * ymax - 33  # convert to y-pos in visual field, 0 -> -20
xmax = b * xmax - 135 # convert to x-pos in visual field, 0 -> -135

fig, axs = plt.subplots(1, 2, figsize=(8, 3))
for i in range(2):
    axs[i].scatter(ypos, xpos, c=xmax if i==0 else ymax,
                   s=20, cmap='gist_ncar',
                   vmin=-135 if i==0 else -40, vmax=5 if i==0 else 40)
    plt.colorbar(axs[i].collections[0], ax=axs[i],
                 label='azimuth (deg)' if i==0 else 'elevation (deg)')
    axs[i].set_xlabel('x-pos (um)')
    axs[i].set_ylabel('y-pos (um)')
    axs[i].axis('equal')
    axs[i].invert_xaxis()
plt.tight_layout()

We can see that the receptive field centers vary continuously over the recording area.

## 3.3 Quantifying model performance

Let's quantify how well this linear model captures the neural activity. For this we will calculate the prediction on held-out test data:

$$ \hat Y_\text{test} = X_\text{test} A .$$

We will compute the variance explained of the predicted test responses to the held-out test images, which is the total variance minus the unexplained - or residual - variance. We will use all 10 repeats which is equivalent to averaging the residuals across these trials:

$$ \text{var. exp.} = \text{var}[Y] - \mathbb{E}_j [(Y - \hat{y})^2]$$

The fraction of variance explained is the ratio of the variance explained and the total variance:

$$ \text{FVE} = \frac{\text{var}[Y] - \mathbb{E}_j [(Y - \hat{y})^2]}{\text{var}[Y]} = 1 - \frac{\mathbb{E}_j [(Y - \hat{y})^2]}{\text{var}[Y]} $$

In [ ]:
# prediction on test data
Y_pred_linear = X_test @ A

################################################################################
# TODO for students: compute the residual variance and fraction of variance explained (FEV)
raise NotImplementedError("Student exercise: compute residual variance and FEV")
################################################################################
# Hint:
# - Y_pred_linear has shape (n_images, n_neurons)
# - Y_test has shape (n_images, n_trials, n_neurons)
# - Broadcast Y_pred_linear across trials to match Y_test shape
# - Compute residual variance: mean squared error between prediction and actual
# - Then compute FEV: 1 - residual / total variance
residual = ...
varexp = ...

# add back bias term (will make our predictions comparable with later sections)
Y_pred_linear += spks.mean(axis=0)

print(f'mean frac varexp (per trial) = {varexp.mean():.3f}')

But we do not expect to explain *all* of the variance with a model of the stimulus responses - we know neurons are modulated by other variables including behaviors like running. Thus, we want to compute the fraction of *explainable* variance explained by taking the ratio between the variance explained and the explainable variance:

$$ FEVE = \frac{\text{var}[Y] - \mathbb{E}_j [(Y - \hat{\mathbf{y}})^2]}{\text{var}[Y] - \sigma^2_{\text{noise}}} = 1 - \frac{\mathbf{E}_j[(Y - \hat{\mathbf{y}})^2] - \sigma^2_\text{noise}}{\text{var}[Y] - \sigma^2_\text{noise}}.$$


In [ ]:
total_var = Y_test.var(axis=(0,1), ddof=1)

################################################################################
# TODO for students: compute noise variance and FEVE
raise NotImplementedError("Student exercise: compute noise_var and FEVE_linear")
################################################################################
# Hint:
# - noise_var: variance across trials (per stimulus), then average across stimuli
# - Use ddof=1 for unbiased estimate
noise_var = ...
FEVE_linear = ...

plt.figure(figsize=(4,3))
plt.hist(FEVE_linear, bins=20, color='lightgray', edgecolor='black')
plt.xlabel('FEVE')
plt.ylabel('# of neurons')

print(f'FEVE (average +/- std): {FEVE_linear.mean():.3f} +/- {FEVE_linear.std():.3f}')

---

# 4. Hand-Crafted CNN Filters ⏱️ ~30-40 min  

We found that relatively simple CNNs can model neural responses well. In this section, we explain how convolutional layers work and build fixed filters (e.g., edge detectors), visualize their outputs, and correlate their activations with neural responses.


## 4.1 2D convolution operation

A 2D convolution is an integral of the product of a filter $f$ and an input image $I$ computed at various positions as the filter is slid across the input. The output of the convolution operation at position $(x,y)$ can be written as follows, where the filter $f$ is size $(2K+1, 2K+1)$:

\begin{equation}
C(x,y) = \sum_{k_x=-K}^{K} \sum_{k_y=-K}^{K} f(k_x,k_y) I(x+k_x,y+k_y)
\end{equation}

This **convolutional filter** is often called a **kernel**.

Here is an illustration of a 2D convolution from this [article](https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-convolution-neural-networks-e3f054dd5daa):

In [ ]:
# @markdown Execute this cell to view convolution gif

from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*5BwZUqAqFFP5f3wKYQ6wJg.gif', width=600)

## 4.2 2D convolutions in deep learning

A 2D convolutional layer has multiple output channels. Each output **channel** is the result of a 2D convolutional filter applied to the input. In the gif below, the input is in blue, the filter is in gray, and the output is in green. The number of units in the output channel depends on the *stride* you set. In the gif below, the stride is 1 because the input image is sampled at each position, a stride of 2 would mean skipping over input positions. In most applications, especially with small filter sizes, a stride of 1 is used.

*Technical note*: if kernel size is odd and you set the padding to half the kernel size and the stride to 1 (as is shown below), you get a **channel** of units that is the same size as the input.

In [ ]:
# @markdown Execute this cell to view convolution gif

from IPython.display import Image
Image(url='https://miro.medium.com/max/790/1*1okwhewf5KCtIPaFib4XaA.gif', width=300)

When we fit the linear receptive fields of neurons, we fit a matrix of size number of pixels by number of neurons. This is equivalent to a fully connected - or linear layer - in a neural network.

Convolutional layers are different from their fully connected counterparts in two ways:
  * In a fully connected layer, each unit computes a weighted sum over all the input units. In a convolutional layer, on the other hand, each unit computes a weighted sum over only a small patch of the input, referred to as the unit's **receptive field**.
  * In a fully connected layer, each unit uses its own independent set of weights to compute the weighted sum. In a convolutional layer, all the units (within the same channel) **share the same weights**. This set of shared weights is called the **convolutional filter or kernel**. The result of this computation is a convolution, where each unit has computed the same weighted sum over a different part of the input. This reduces the number of parameters in the network substantially.

<p align="center">
  <img src="https://github.com/NeuromatchAcademy/course-content/blob/main/tutorials/static/weight-sharing.png?raw=true" width="600" />
</p>

## 4.3 2D convolutions in Pytorch

Now let's implement 2D convolutional operations. We will use multiple convolutional channels and implement this operation efficiently using pytorch. A *layer* of convolutional channels can be implemented with one line of code using the PyTorch class `nn.Conv2d()`, which requires the following arguments for initialization (see full documentation [here](https://pytorch.org/docs/master/generated/torch.nn.Conv2d.html)):
  * $C^{in}$: the number of input channels
  * $C^{out}$: the number of output channels (number of different convolutional filters)
  * $K$: the size of the $C^{out}$ different convolutional filters

When you run the network, you can input a stimulus of arbitrary size $(H^{in}, W^{in})$, but it needs to be shaped as a 4D input $(N, C^{in}, H^{in}, W^{in})$, where $N$ is the number of images. In our case, $C^{in}=1$ because there is only one color channel (our images are grayscale, but often $C^{in}=3$ in image processing).

In [ ]:
class ConvolutionalLayer(nn.Module):
  """ One convolutional layer
     Attributes: conv (nn.Conv2d): convolutional layer
  """
  def __init__(self, c_in=1, c_out=8, K=7, filters=None):
    """Initialize layer

    Args:
        c_in: number of input stimulus channels
        c_out: number of output convolutional channels
        K: size of each convolutional filter
        filters: (optional) initialize the convolutional weights

    """
    super().__init__()
    self.conv = nn.Conv2d(c_in, c_out, kernel_size=K, padding=K//2, stride=1)
    if filters is not None:
      self.conv.weight = nn.Parameter(filters)
      self.conv.bias = nn.Parameter(torch.zeros((c_out,), dtype=torch.float32))

  def forward(self, s):
    """Run stimulus through convolutional layer

    Args:
        s (torch.Tensor): n_stimuli x c_in x h x w tensor with stimuli

    Returns:
        (torch.Tensor): n_stimuli x c_out x h x w tensor with convolutional layer unit activations.

    """
    a = self.conv(s)  # output of convolutional layer

    return a

See that `ConvolutionalLayer` takes as input `filters`. We have predesigned some filters that you can use by calling the `filters` function below. These are similar to filters we think are implemented in biological circuits such as the retina and the visual cortex. The first two are **center-surround** filters and the next six are **gabor** filters. Check out this [website](http://www.cns.nyu.edu/~david/courses/perception/lecturenotes/ganglion/ganglion.html) for more details on center-surround filters, and this [website](https://en.wikipedia.org/wiki/Gabor_filter) for more details on gabor filters, if you're interested.



In [ ]:
# make filters
example_filters = filters(out_channels=8, K=25)

# visualize filters
fig, axs = plt.subplots(1, 8, figsize=(12,4))
for i in range(len(example_filters)):
    ax = axs[i]
    show_stimulus(example_filters[i].squeeze(), ax=ax, filters=True)
    ax.set_title(f'filter {i}')
    ax.axis('off')

We will now run the convolutional layer on our stimulus. We will apply the convolution filters on the natural images with size 66x130.

Reminder, `nn.Conv2d` takes in a tensor of size $(N, C^{in}, H^{in}, W^{in}$) where $N$ is the number of stimuli, $C^{in}$ is the number of input channels, and $(H^{in}, W^{in})$ is the size of the stimulus. We will need to add the channel dimension to our stimulus, then input it to the convolutional layer.

In [ ]:
# apply the filters on the images
# Stimulus parameters
in_channels = 1  # how many input channels in our images
nstim = 5

# select stimuli
np.random.seed(1)
example_stims = np.random.choice(imgs.shape[0], nstim, replace=False)
stimuli = torch.from_numpy(imgs[example_stims])
stimuli = stimuli.unsqueeze(1)  # add channel dimension
print('input stimuli shape:', stimuli.shape)

# Convolution layer parameters
K = 25  # filter size
out_channels = 8  # how many convolutional channels to have in our layer
example_filters = filters(out_channels, K)  # create filters to use


################################################################################
## TODO for students: create convolutional layer in pytorch
# Complete and uncomment
raise NotImplementedError("Student exercise: create convolutional layer")
################################################################################

# Initialize conv layer and add weights from function filters
# you need to specify :
# * the number of input channels c_in
# * the number of output channels c_out
# * the filter size K
convLayer = ConvolutionalLayer(..., filters=example_filters)


convout = convLayer(stimuli)  # run stimuli through convolutional layer
convout = convout.detach().cpu().numpy()  # convert to numpy
print('convolutional output shape:', convout.shape)

Next we visualize the outputs of the convolution. `convout` is a tensor of size $(N, C^{out}, H^{in}, W^{in})$ where $N$ is the number of examples and $C^{out}$ are the number of convolutional channels. It is the same size as the input because we used a stride of 1 and padding that is half the kernel size.


In [ ]:
plot_example_activations(stimuli, convout, example_filters,
                         channels=np.arange(0, out_channels), vmax=30)

**Q: Do you notice any patterns across the activations from the Gabor filters? What would happen if we summed the absolute value of the Gabor filters with the same orientation but different phase?**

For each location (x, y) on the input image, one filter is applied to compute the output at that location. We can calculate the correlation of the values of each filter at each position with the neural responses.

In [ ]:
# get test images
imgs_test = imgs[istim_test]
print('test images shape:', imgs_test.shape)

# average responses across trials
spks_test_avg = spks_test.mean(axis=1)
print('average test responses shape:', spks_test_avg.shape)

# compute convolutional layer responses to all test images
stimuli = torch.from_numpy(imgs_test).unsqueeze(1) # add channel dimension
convout_test = convLayer(stimuli).detach().cpu().numpy()
print('convolutional output shape:', convout_test.shape)

# calculate correlation of convolutional layer responses with neural responses
convout_test = convout_test.reshape(convout_test.shape[0], -1)

# correlation is matrix multiplication of z-scored neural responses and z-scored convolutional layer responses
spks_test_avg -= spks_test_avg.mean(axis=0)
spks_test_avg /= spks_test_avg.std(axis=0)
convout_test -= convout_test.mean(axis=0)
convout_test /= convout_test.std(axis=0)

################################################################################
# TODO for students: calculate the correlation between the activity and 
# the features from the conv layer.
raise NotImplementedError("Student exercise: compute correlation between activity and conv features")
################################################################################

# Hint:
# - spks_test_avg has shape (n_images, n_neurons)
# - convout_test has shape (n_images, n_channels)
# - You want a (n_neurons × n_channels) correlation matrix
# - Don't forget to divide by the number of images to normalize the dot product
cc = ...

# reshape correlation matrix to (n_neurons, out_channels, h, w)
cc = cc.reshape(spks_test_avg.shape[1], out_channels, h, w)

print('correlation shape:', cc.shape)

We can visualize the correlations with each convolutional channel at each position for a random set of neurons:

In [ ]:
ineurons = np.random.choice(cc.shape[0], 5, replace=False)
plot_correlation_maps(spks_test[:, :, ineurons], cc[ineurons], example_filters)

We can see that neurons often have higher correlations with activations in a certain part of the image, across maps. This is consistent with what we saw with the linear receptive fields: neurons are receiving visual input from a subset of space.

---

# 5. One-layer CNN (learnable filters + readout layer) ⏱️ ~1 hr  

Now we will build a model to predict the neural activity using a convolutional layer. On top of this convolutional layer we add a readout layer that learns the spatial pooling and feature weighting for each neuron. The convolutional layer will be the same for all neurons, and the readout will have specific weights for each neuron.

<img src="https://github.com/MouseLand/minimodel/blob/main/figures/model_structure/1layermodel.png?raw=true" 
     alt="one-layer model structure" 
     height="300"/>

## 5.1 Neuron-specific readout weights

We can see that in the above example we have 8 x 66 x 130 activations in the output of the convolutional layer, which is 68,640 outputs in total. If we use a fully-connected linear layer here, then we will need to fit 68,640 parameters per neuron. Instead we make some simplifying assumptions about the complexity of the spatial and feature pooling that the neuron does.

First, we assume that the neuron pools spatially with all positive pooling weights, and that these spatial pooling weights can be approximated by a rank-1 multiplication:

$$ W_{yx} = \mathbf{w}_y \mathbf{w}_x^\top$$

In [ ]:
## spatial pooling weights

# make example spatial pooling weights from gaussian filters
w_y, w_x = np.zeros(h), np.zeros(w)
w_y[h//2] = 1
w_x[w//2] = 1
# can vary width of gaussians to change the size/shape
sigma_y, sigma_x = 8, 12
w_y = gaussian_filter(w_y, sigma_y)
w_x = gaussian_filter(w_x, sigma_x)

################################################################################
# TODO for students: compute the outer product of w_y and w_x
raise NotImplementedError("Student exercise: compute spatial pooling matrix W_yx")
################################################################################
# Hint:
# - Use np.outer to create a 2D spatial pooling matrix from 1D vertical and horizontal weights
# - This simulates a rank-1 spatial readout for a neuron
W_yx = ...

plt.figure(figsize=(3,1))
plt.imshow(W_yx, cmap='gray');


**Q: How does this rank-1 assumption restrict the shape of the spatial pooling weights?**

The next assumption we make is that all features (channels) have the same spatial pooling in the readout, meaning we have one $W_{yx}$ for each neuron, that is combined with the feature weights per channel $\mathbf{w}_c$, as an outer product of all three vectors:

$$ W_{cyx} = [w^1_c (\mathbf{w}_y \mathbf{w}_x^\top) \, \cdots \, w^{N_c}_c (\mathbf{w}_y \mathbf{w}_x^\top)] $$

where $N_c$ is the number of output channels in the convolutional layer.

In the example above, this reduces the number of parameters in each neuron's readout from $HWN_c$ (68,640) to $H + W + N_c$ (204).

## 5.2 Combining conv layer with readout

We next put these two parts together into a single model, and fit the weights of the model directly to the neural activity. We will fit this model from scratch, not initializing the filters.

In [ ]:
class PredModel(nn.Module):
  """ Model to predict neural responses w/ 1 conv layer and neuron-specific readouts
     Attributes:
        conv (nn.Conv2d): convolutional layer
        W_y (torch.Tensor): readout weights over height of image, shape (n_neurons, h)
        W_x (torch.Tensor): readout weights over width of image, shape (n_neurons, w)
        W_c (torch.Tensor): readout weights over convolutional channels, shape (n_neurons, c_out)
  """
  def __init__(self, c_in=1, c_out=32, K=13, n_neurons=1000,
                h_in=33, w_in=65):
    """ Initialize model

    Args:
        c_in: number of input stimulus channels
        c_out: number of output convolutional channels
        K: size of each convolutional filter
        n_neurons: number of neurons to predict
        h_in: height of input stimulus
        w_in: width of input stimulus
        maxpool: whether to apply max pooling after convolutional layer
    """
    super().__init__()
    self.conv = nn.Conv2d(c_in, c_out, kernel_size=K, padding=K//2, stride=1)
    self.activation = nn.ELU()  # activation function
    self.Wy = nn.Parameter(0.01 * torch.randn((n_neurons, h_in)))  # readout weights over height
    self.Wx = nn.Parameter(0.01 * torch.randn((n_neurons, w_in)))  # readout weights over width
    self.Wc = nn.Parameter(0.01 * torch.randn((n_neurons, c_out)))  # readout weights over convolutional channels


  def forward(self, s):
    """Run stimulus through convolutional layer and readout

    Args:
        s (torch.Tensor): n_stimuli x c_in x h x w tensor with stimuli

    Returns:
        (torch.Tensor): n_stimuli x n_neurons tensor with predicted neural responses

    """
    a = self.conv(s)  # output of convolutional layer
    a = self.activation(a)
    
    ################################################################################
    # TODO for students: use torch.einsum to implement the outer-product readout
    # Hint:
    # - a has shape (n_stimuli, c_out, h, w): conv output
    # - Wy, Wx, Wc have shapes (n_neurons, h), (n_neurons, w), (n_neurons, c_out)
    # - Output should be (n_stimuli, n_neurons): prediction per stimulus per neuron
    raise NotImplementedError("Student exercise: apply factorized spatial-channel readout using einsum")
    ################################################################################
    pred = ...

    pred = self.activation(pred)
    pred += 1 + 1e-12 # shift prediction to be all positive for ELU

    return pred


# we will downsample the images again to speed up training
b = 2
model = PredModel(c_in=in_channels, c_out=out_channels, K=K, n_neurons=n_neurons,
                  h_in=h//b, w_in=w//b)#.to(device)

print(model)

# print model parameters
print('trainable parameters:')
print([(p[0], p[1].shape) for p in model.named_parameters()])

## 5.3 Training the model

We will use the **AdamW** optimizer to modify our weights to reduce the loss function, which consists of iterating three steps.

1. **Evaluate the loss** on the training data in batches
```python
Y_pred = model(X_batch)
loss = loss_fn(Y_pred, Y_batch)
```
where `X_batch` are a batch of training images, and `Y_batch` are the neuron responses to a batch of training images (tensor of number of stimuli by number of neurons).

We will use the Poisson loss introduced in the Day 1 tutorial:
```python
loss = (Y_pred - Y_batch * torch.log(Y_pred)).sum(axis=0).mean()
```

2. **Compute the gradient of the loss** with respect to each of the network weights, with the `.backward()` method of the loss `loss`. Note that the gradients of each parameter need to be cleared before calling `.backward()`, otherwise the gradients will accumulate across iterations, with `.zero_grad()`:
```python
optimizer.zero_grad()
loss.backward()
```

3. **Update the network weights** by descending the gradient. In Pytorch, we can do this using built-in optimizers. We'll use the `optim.AdamW` optimizer (documentation [here](https://pytorch.org/docs/stable/optim.html#torch.optim.AdamW)) which updates parameters along the negative gradient, scaled by a learning rate and scaled by the moving average of the gradients, and with scaled momentum. To optimize *all* the parameters of a network `model` using a learning rate of .001, the optimizer would be initialized as follows
  ```python
  optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.1)
  ```
  
  We can also specify learning rates and weight decay values per parameter sets like:
  ```python
  optimizer = torch.optim.AdamW([{'params': model.conv.parameters(), 'weight_decay': 0.1},
                                 {'params': [model.Wy, model.Wx], 'weight_decay': 1.0},
                                 {'params': model.Wc, 'weight_decay': 0.1},
                                  ], lr=0.001)
  ```


  After computing all the parameter gradients in step 2, we can then update each of these parameters using the `.step()` method of this optimizer,
  ```python
  optimizer.step()
  ```

This optimizer [AdamW](https://pytorch.org/docs/stable/optim.html#torch.optim.AdamW) includes `weight_decay` which is an L2 regularization of the weights (reduces the magnitude of the weights on each iteration). Regularization is quite useful with noisy neural data, as we also saw with the linear regression.

In [ ]:
## define train and test data

# train and test images
X_train = imgs[istim_train].copy()
X_train = X_train.reshape(X_train.shape[0], h//b, b, w//b, b).mean(axis=(2,4))  # downsample by 2
X_test = imgs[istim_test].copy()
X_test = X_test.reshape(X_test.shape[0], h//b, b, w//b, b).mean(axis=(2,4))  # downsample by 2

# normalize neural activity by standard deviation
Y_train = spks.copy()
Y_train /= spks.std(axis=0)  # divide by std neuron response
Y_test = spks_test.copy()
Y_test /= spks.std(axis=0)  # divide by std neuron response

# use one repeat for test loss (so it is similar to training loss)
Y_test0 = Y_test[:,0]

Start training the model, then maybe get a coffee ☕ it will take around 2-3 min w/ 100 neurons.

In [ ]:
# put data on GPU (if data is too big, this should be done on each batch)
Y_train_t = torch.from_numpy(Y_train).to(device)
Y_test0_t = torch.from_numpy(Y_test0).to(device)
X_train_t = torch.from_numpy(X_train).unsqueeze(1).to(device)
X_test_t = torch.from_numpy(X_test).unsqueeze(1).to(device)

# declare model here to reinitialize as needed before training
nconv1 = 32 # we will have 32 trainable filters
model = PredModel(c_in=1, c_out=nconv1, K=13, n_neurons=n_neurons).to(device)

# initialize PyTorch AdamW optimizer
learning_rate = 5e-3
optimizer = torch.optim.AdamW([{'params': model.conv.parameters(), 'weight_decay': 0.1},
                                {'params': [model.Wy, model.Wx], 'weight_decay': 1},
                                {'params': model.Wc, 'weight_decay': 0.1},
                                ], lr=learning_rate)

# Loop over epochs
n_epochs = 60
batch_size = 100
train_loss, test_loss = np.zeros(n_epochs), np.nan * np.zeros(n_epochs)
nstim, nstim_test = X_train.shape[0], X_test.shape[0]

tic = time.time()
for iepoch in range(n_epochs):
    model.train() # set model to training mode
    iperm = np.random.permutation(nstim)  # random permutation of stimuli
    if iepoch % 15 == 0 and iepoch > 0:
        learning_rate /= 3 # reduce learning rate by factor of 3 every 15 epochs
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    for istart in range(0, nstim, batch_size):
        X_batch = X_train_t[iperm[istart : istart + batch_size]]
        Y_batch = Y_train_t[iperm[istart : istart + batch_size]]
        # compute network output from inputs in train_data
        Y_pred = model(X_batch)

        # evaluate loss function using the possion loss
        loss = (Y_pred - Y_batch * torch.log(Y_pred)).sum(axis=0).mean()

        optimizer.zero_grad() # clear previous gradients
        loss.backward() # compute gradients
        optimizer.step() # update weights

        train_loss[iepoch] += loss.item() # average loss over epoch

        # clamp Wy and Wx to be non-negative
        model.Wy.data.clamp_(0)
        model.Wx.data.clamp_(0)

    train_loss[iepoch] /= nstim

    # evaluate test loss
    model.eval()
    test_loss[iepoch] = 0.0
    Y_pred_test = np.zeros((nstim_test, n_neurons), 'float32')
    with torch.no_grad():
        for istart in range(0, nstim_test, batch_size):
            X_batch = X_test_t[istart : istart + batch_size]
            Y_batch = Y_test0_t[istart : istart + batch_size]
            Y_pred = model(X_batch)
            Y_pred_test[istart : istart + batch_size] = Y_pred.cpu().numpy()
            test_loss[iepoch] += (Y_pred - Y_batch * torch.log(Y_pred)).sum(axis=0).mean()
        test_loss[iepoch] /= nstim_test

    print(f'epoch {iepoch}, train_loss = {train_loss[iepoch]:0.4f}, test_loss = {test_loss[iepoch]:0.4f}, LR = {learning_rate:.1e}, time {time.time()-tic:.2f}s')

# remove to free GPU memory
del Y_train_t, Y_test0_t, X_train_t, X_test_t

# save model
model_path = os.path.join(weight_path, 'conv_model.pth')
torch.save(model.state_dict(), model_path)

Quantify performance with variance explained and FEVE:

In [ ]:
Y_pred_conv1 = Y_pred_test

# variance explained per neuron - across stimuli and repeats
residual = ((Y_pred_conv1[:,np.newaxis] - Y_test)**2).mean(axis=(0,1))
varexp = 1 - residual / Y_test.var(axis=(0,1))

print(f'mean frac varexp (per trial) = {varexp.mean():.3f}')

total_var = Y_test.var(axis=(0,1), ddof=1)
noise_var = Y_test.var(axis=1, ddof=1).mean(axis=0)
FEVE_conv1 = 1 - (residual - noise_var) / (total_var - noise_var)

print(f'FEVE (average +/- std): {FEVE_conv1.mean():.3f} +/- {FEVE_conv1.std():.3f}')

fig, axs = plt.subplots(1, 2, figsize=(8,3))
axs[0].hist(FEVE_conv1, bins=20, color='lightgray', edgecolor='black')
axs[0].set_xlabel('FEVE')
axs[0].set_ylabel('# of neurons')

# compare to linear model
axs[1].scatter(FEVE_linear, FEVE_conv1, s=10, c='b', alpha=0.5)
axs[1].scatter(FEVE_linear.mean(), FEVE_conv1.mean(), c='r', marker='*', s=100)
axs[1].plot([0, 1], [0, 1], 'k--', lw=2)
axs[1].set_xlabel('linear model')
axs[1].set_ylabel('conv+readout model')
axs[1].axis('square')
axs[1].set_title('FEVE comparison')
plt.tight_layout()

We have more than doubled our FEVE now with this fairly simple model.

(optional) try changing the number of convolutional filters in the first layer (`nconv1`), and quantify performance.

(optional) remove the clamping of the readout weights - allowing them to be positive and negative, and quantify performance.

## 5.4 Visualizing the model

We have the fully-learned first convolutional layer and the readout learned for each neuron.

First we will look at the convolutional filters:

In [ ]:
conv_filters = model.conv.weight.data.detach().cpu().numpy()

rm_model = Rastermap(n_PCs=16, n_clusters=None).fit(conv_filters.reshape(conv_filters.shape[0], -1))
isort_filt = rm_model.embedding[:,0].argsort()
conv_filters = conv_filters[isort_filt]

# visualize filters
fig, axs = plt.subplots(4, 8, figsize=(12,6))
for i in range(len(conv_filters)):
    ax = axs[i // 8, i % 8]
    show_stimulus(conv_filters[i].squeeze(), ax=ax, filters=True, vmax=5e-1)
    ax.axis('off')

**Q: What sorts of features do you see in these filters?**

Now we'll visualize the readout weights for a subset of neurons. Here we plot  $W_{yx} = \mathbf{w}_y \mathbf{w}_x^\top$:

In [ ]:
Wy = model.Wy.data.detach().cpu().numpy()
Wx = model.Wx.data.detach().cpu().numpy()
fig, axs = plt.subplots(10, 10, figsize=(12, 8))
for i in range(100):
    ax = axs[i // 10, i % 10]
    ax.imshow(np.outer(Wy[i], Wx[i]), cmap='gray', vmin=0, vmax=5e-3)
    ax.axis('off')

Note that the spatial readout weights are all positive, forcing the model to perform a positive pooling over pixels across channels. The **sign and strength** of each channel's contribution are instead controlled by the **channel weights**, which can be either positive or negative.

The channel weights are neurons by channels, we will sort the neurons and channels by rastermap for visualization:

In [ ]:
Wc = model.Wc.data.detach().cpu().numpy()

# sort neurons
rm_model = Rastermap(n_PCs=16, n_clusters=50).fit(Wc)
Wc_sort = Wc[rm_model.embedding[:, 0].argsort()]

# sort by filters
Wc_sort = Wc_sort[:, isort_filt]

# visualize Wc and filters
plot_Wc(Wc_sort, conv_filters, vmax_filt=0.5, vmax_Wc=0.5)


We have sorted the neurons by the similarities of their $\mathbf{w}_c$ vectors. Even so we can many different patterns of $\mathbf{w}_c$ across the population, suggesting that there are many different combinations of convolutional channels.

---

# 6. Two-layer CNN (“Full Model” for all the recorded neurons) ⏱️ ~20-30 min  

We found in [Du et al](https://www.nature.com/articles/s41467-025-61171-9) that a two-layer CNN (with 16 and 320 channels) is sufficient to match more complex architectures. Here, we build and train this full model and visualize the learned weights.

<img src="https://github.com/MouseLand/minimodel/blob/main/figures/model_structure/fullmodel.png?raw=true" 
     alt="fullmodel structure" 
     height="300"/>

Here we build the model, specifying the number of convolutional layers `nlayers`, and the number of output channels per layer `nconv1` and `nconv2`.

In [ ]:
# build model
from minimodel import model_builder
nlayers = 2
nconv1 = 16
nconv2 = 320
fullmodel, in_channels = model_builder.build_model(NN=n_neurons, n_layers=nlayers, n_conv=nconv1, n_conv_mid=nconv2)
model_name = model_builder.create_model_name(data.mouse_names[mouse_id], data.exp_date[mouse_id], n_layers=nlayers, in_channels=in_channels)

model_path = os.path.join(weight_path, 'fullmodel', model_name)
fullmodel = fullmodel.to(device)

print(fullmodel)

## 6.1 Load / train the model

On the T4, the full training will take over 1 hour, so we will load the checkpoint by default.

In [ ]:
# train model
from minimodel import model_trainer

do_train = False # whether or not to train the model
if do_train:
    # split into train and validation set
    itrain, ival = data.split_train_val(istim_train, train_frac=0.9)
    spks_val = torch.from_numpy(Y_train[ival]).to(device)
    spks_train = torch.from_numpy(Y_train[itrain]).to(device)
    img_train = torch.from_numpy(imgs[istim_train][itrain]).to(device).unsqueeze(1)
    img_val = torch.from_numpy(imgs[istim_train][ival]).to(device).unsqueeze(1)
    print("img sizes: [train], [val]")
    print(img_train.shape, img_val.shape)
    best_state_dict = model_trainer.train(fullmodel, spks_train, spks_val, img_train, img_val, device=device)
    torch.save(best_state_dict, model_path)
    print('saved model', model_path)
else:
    fullmodel, in_channels = model_builder.build_model(NN=n_neurons_all, n_layers=nlayers, n_conv=nconv1, n_conv_mid=nconv2)
    fullmodel = fullmodel.to(device)

fullmodel.load_state_dict(torch.load(model_path), strict=False)
print('loaded model', model_path)

Now we quantify the model performance:

In [ ]:
# compute model FEVE
img_test = torch.from_numpy(imgs[istim_test]).to(device).unsqueeze(1)
test_pred = model_trainer.test_epoch(fullmodel, img_test)
Y_pred_fullmodel = test_pred[:, igood]

residual = ((Y_pred_fullmodel[:,np.newaxis] - Y_test)**2).mean(axis=(0,1))
varexp = 1 - residual / Y_test.var(axis=(0,1))

print(f'mean frac varexp (per trial) = {varexp.mean():.3f}')

FEVE_fullmodel = 1 - (residual - noise_var) / (total_var - noise_var)

print(f'FEVE (average +/- std): {FEVE_fullmodel.mean():.3f} +/- {FEVE_fullmodel.std():.3f}')

fig, axs = plt.subplots(1, 2, figsize=(8,3))
axs[0].hist(FEVE_fullmodel, bins=20, color='lightgray', edgecolor='black')
axs[0].set_xlabel('FEVE')
axs[0].set_ylabel('# of neurons')

# compare to linear model
axs[1].scatter(FEVE_conv1, FEVE_fullmodel, s=10, c='b', alpha=0.5)
axs[1].scatter(FEVE_conv1.mean(), FEVE_fullmodel.mean(), c='r', marker='*', s=100)
axs[1].plot([0, 1], [0, 1], 'k--', lw=2)
axs[1].set_ylabel('16-320+readout model')
axs[1].set_xlabel('conv+readout model')
axs[1].axis('square')
axs[1].set_title('FEVE comparison')
plt.tight_layout()


We explain even more variance with this two-layer model.

We can look at `Y_pred_linear`, `Y_pred_conv1` and `Y_pred_fullmodel` for a few example neurons - what differences do you see?

In [ ]:
k = 0 # neuron index

plot_predictions(Y_test, Y_pred_linear, Y_pred_conv1, Y_pred_fullmodel, k=k)

## 6.2 Visualize conv1

As in the previous model, we can easily visualize the first layer of convolutional filters:

In [ ]:
################################################################################
# TODO for students: try to find the conv1 weights name from the model
# Hint:
# - Print the fullmodel structure to inspect the module hierarchy
# - Look for the first conv layer under fullmodel.core.features
raise NotImplementedError("Student exercise: extract conv1 weights from fullmodel")
################################################################################

conv1_w = ...  # get conv1 weights from fullmodel

print('conv1_w shape:', conv1_w.shape)
isort = [14,3,4,5,15,0,12,11,10,6,7,8,9,13,1,2]
fig, ax = plt.subplots(4, 4, figsize=(4, 4))
for i in range(4):
    for j in range(4):
        idx = i * 4 + j
        ax[i, j].imshow(conv1_w[isort[idx]], cmap='coolwarm', vmin=-0.2, vmax=0.2)
        ax[i, j].axis('off')

**Q: What do these filters look like?**

## 6.3 Visualize the conv2

The second convolutional layer is a **depthwise-separable** convolutional layer, which consists of two types of operations:

1. **1×1 convolution**: Since the filter size is 1×1, the weights are of size input channels by output channels. The operation is equivalent to a linear operation, where each (x, y) location is weighted and summed across the input channels to create each output channel. This means for our 16 input channels and 320 output channels, we have 16x320 weights.

2. **Spatial-only convolution**: operates over the **local spatial neighborhood** of each pixel, separately for each input channel. This means for our 9x9 spatial filters and 320 input channels, we have 9x9x320 weights.

Why this 1×1 → spatial conv structure?

This structure still allows for combining both across features (1x1 conv) and across pixels (spatial conv) which reducing the total number of parameters and operations that the network needs to perform. This increases efficiency and reduces network size.

In [ ]:
conv2_1x1 = fullmodel.core.features.layer1.ds_conv.in_depth_conv.weight.data.cpu().numpy().squeeze()
print('conv2_1x1 shape:', conv2_1x1.shape)
conv2_spatial = fullmodel.core.features.layer1.ds_conv.spatial_conv.weight.data.cpu().numpy().squeeze()
print('conv2_spatial shape:', conv2_spatial.shape)
conv2_out = fullmodel.core.features.layer1.ds_conv.out_depth_conv.weight.data.cpu().numpy().squeeze()
print('conv2_out shape:', conv2_out.shape)

**Q: How does each channel in the second layer relate to the first layer? How much of a reduction in parameters does this depthwise separable convolution have versus a standard convolutional layer?**

Let's visualize the 1x1 convolution. Here we plot the 1x1 convolution weights as the rectangle colors from blue to red with their corresponding input conv1 filter. We show these for a few example output channels

In [ ]:
# visualize 1x1 conv
plot_conv2_1x1(conv1_w, conv2_1x1, ichannels=[127,242, 102,68,204,217,293])

Let's visualize the spatial filters to see what the model has learned in the second layer.

In [ ]:
ichannels = [19,275,205,58, 197,313,276,308,129, 20]
fig, axs = plt.subplots(1, 10, figsize=(14, 2))
for i in range(10):
    ax = axs[i]
    ax.imshow(conv2_spatial[ichannels[i]], cmap='RdBu_r', vmin=-0.2, vmax=0.2)
    ax.axis('off')
    ax.set_title(f'channel {ichannels[i]}', fontsize=8)

**Q: What sorts of filters do you see?**

## 6.4 Visualize the readout

As before, each neuron has a set of readout weights consisting of three vectors $\mathbf{w}_y$, $\mathbf{w}_x$, and $\mathbf{w}_c$, which are combined as an outer product.

We again visualize $W_{yx} = \mathbf{w}_y \mathbf{w}_x^\top$

In [ ]:
Wx = fullmodel.readout.Wx.detach().cpu().numpy()[igood]
Wy = fullmodel.readout.Wy.detach().cpu().numpy()[igood]

fig, axs = plt.subplots(10, 10, figsize=(12, 8))
for i in range(100):
    ax = axs[i // 10, i % 10]
    ax.imshow(np.outer(Wy[i], Wx[i]), cmap='gray', vmin=0, vmax=5e-3)
    ax.axis('off')

**Q: How do these $W_{yx}$ matrices compare to the ones that were optimal for the one-layer model?**


Now, let’s visualize the channel weights.

In [ ]:
Wc = fullmodel.readout.Wc.data.detach().cpu().numpy().squeeze()[igood]

# sort neurons
rm_model = Rastermap(n_PCs=32, n_clusters=50).fit(Wc)
Wc_sort = Wc[rm_model.embedding[:, 0].argsort()]

# sort by filters
rm_model = Rastermap(n_PCs=32, n_clusters=50).fit(Wc_sort.T)
Wc_sort = Wc_sort[:, rm_model.embedding[:, 0].argsort()]


# visualize Wc and filters
plot_Wc(Wc_sort, None, vmax_filt=0.5, vmax_Wc=0.5)


From $\mathbf{w}_c$ we can observe that neurons have non-zero weights for many of the channels, using most of the 320 channels, making it hard to interpret the feature selectivity of a single neuron. Also, the neurons each have different patterns of $\mathbf{w}_c$, meaning they are combining these channels in diverse ways, creating a high-dimensional space of image responses.

[insert the fullmodel and minimodel structure]

**Q: Does the neuron have to use all channels? Or it can use a subset of channels?**

---

# 7. Minimodel per Neuron ⏱️ ~45 min  

It appears that the neurons are quite diverse in their feature selectivity. Perhaps we can fit single neuron models which have a smaller second convolutional layer with features that are specific for each neuron. We call these "minimodels".

<img src="https://github.com/MouseLand/minimodel/blob/main/figures/model_structure/minimodel.png?raw=true" 
     alt="minimodel structure" 
     height="300"/>
     
Let's build a simple minimodel for one neuron. First we choose a single neuron (can try different neurons).

To obtain optimal performance, we split our training set into training and validation data, and use the validation data assess performance across training. The best-performing model on the validation data is saved.


In [ ]:
# choose specific neuron in Y_train
ineuron = 150
ineur = [ineuron]

# split neural data into train and val
itrain, ival = data.split_train_val(istim_train, train_frac=0.9)

# put data on GPU (if data is too big, this should be done on each batch)
Y_train_t = torch.from_numpy(Y_train[itrain][:, ineur]).to(device)
Y_val_t = torch.from_numpy(Y_train[ival][:, ineur]).to(device)
Y_test0_t = torch.from_numpy(Y_test0[:, ineur]).to(device)
X_train_t = torch.from_numpy(X_train[itrain]).unsqueeze(1).to(device)
X_val_t = torch.from_numpy(X_train[ival]).unsqueeze(1).to(device)
X_test_t = torch.from_numpy(X_test).unsqueeze(1).to(device)

print('train images shape:', X_train_t.shape)
print('val images shape:', X_val_t.shape)
print('test images shape:', X_test_t.shape)
print('train responses shape:', Y_train_t.shape)
print('val responses shape:', Y_val_t.shape)
print('test responses shape:', Y_test0_t.shape)

Ly, Lx = X_train_t.shape[2], X_train_t.shape[3]

## 7.1 Build minimodel

Here we build the minimodel with two convolutional layers (`nlayers = 2`), with 16 channels in the first conv layer (`nconv1`) as before, and we reduce the second convolutional layer to 64 output channels (`nconv2`). We have an additional regularizer on $\mathbf{w}_c$ in this case, called the Hoyer-Square regularizer, which is a differentiable approximation of an L0 regularizer. We control the strength of this regularizer with `hs_readout`: increasing this value will increase the sparsity of the $\mathbf{w}_c$ weights.

In [ ]:
# build model
from minimodel import model_builder
nlayers = 2
nconv1 = 16
nconv2 = 64

hs_readout = 0.03

model, in_channels = model_builder.build_model(NN=1, n_layers=nlayers, n_conv=nconv1, n_conv_mid=nconv2, Wc_coef=0.2, input_Ly=Ly, input_Lx=Lx, kernel_size=[13,5])

# name model according to structure
model_name = model_builder.create_model_name(data.mouse_names[mouse_id], data.exp_date[mouse_id],
                                             ineuron=ineur[0], n_layers=nlayers, in_channels=in_channels, hs_readout=hs_readout)
model_path = os.path.join(weight_path, model_name)
model = model.to(device)

print(model)

## 7.2 Train minimodel

We will train the minimodel, with this [function](https://github.com/MouseLand/minimodel/blob/a917a4abd9c799ad4709c561e9587c1b8a41d03f/minimodel/model_trainer.py#L75). We initialize the first layer convolutional filters with the full model filters, and we turn off gradient computation for these filters - keeping their weights fixed during training.

To speed up training, we will use the downsampled images as in previous sections. The full model was trained on the full images so we need to downsample the `conv1` filters.

In [ ]:
import cv2
# Extract and downsample conv1 filters from the full model
full_conv1 = fullmodel.core.features.layer0.conv.weight.data.cpu().numpy()
print("Original shape:", full_conv1.shape)

ks = 13  # target kernel size
conv1_w = np.array([
    cv2.resize(filt[0], (ks, ks))[None, ...]
    for filt in full_conv1
])

print("Downsampled shape:", conv1_w.shape)

Train the model, will take < 1 min.

In [ ]:
# train model
from minimodel import model_trainer
if not os.path.exists(model_path):
    ################################################################################
    # TODO for students: initialize conv1 filters from the full model
    # and prevent them from being updated during training
    raise NotImplementedError("Student exercise: initialize and freeze conv1 filters")
    ################################################################################
    # Hint:
    # - Assign conv1_w to the conv1 weight of the model
    # - Then set requires_grad = False for that same parameter
    ... = torch.from_numpy(conv1_w).to(device, dtype=torch.float)
    ... = False

    # train the minimodel
    best_state_dict = model_trainer.train(model, Y_train_t, Y_val_t, X_train_t, X_val_t, device=device, l2_readout=0.2, hs_readout=hs_readout)
    torch.save(best_state_dict, model_path)
    print('saved model', model_path)
model.load_state_dict(torch.load(model_path))
print('loaded model', model_path)

Now we quantify performance:

In [ ]:
# compute model FEVE
test_pred = model_trainer.test_epoch(model, X_test_t)
Y_pred_minimodel = test_pred.copy()

residual = ((Y_pred_minimodel[:,np.newaxis] - Y_test[:,:,ineur])**2).mean(axis=(0,1))
varexp = 1 - residual / Y_test[:,:,ineur].var(axis=(0,1))

print(f'frac varexp (per trial) = {varexp.mean():.3f}')

feve_minimodel = 1 - (residual - noise_var[ineur]) / (total_var[ineur] - noise_var[ineur])

print(f'neuron {ineur[0]}, FEVE minimodel = {feve_minimodel.squeeze():.3f}, FEVE fullmodel = {FEVE_fullmodel[ineur].squeeze():.3f}')


## 7.3 Visualize conv2 in minimodel

In the minimodel, we are reusing the conv1 from the fullmodel, so let's visualize the filters in conv2. Recall that we applied a sparsity penalty to the channel readout weights ($\mathbf{w}_c$), encouraging the model to rely on only a small subset of channels. So we first will see which conv2 have large $\mathbf{w}_c$ weights:

In [ ]:
Wc = model.readout.Wc.cpu().detach().numpy().squeeze()
print('Wc shape:', Wc.shape)

fig, ax = plt.subplots(1, 1, figsize=(4, 3))
ax.plot(np.sort(Wc), linewidth=2)
ax.set_title(f'Wc for neuron {ineuron}')
# set right and top spines invisible
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

We can see that some channels now have *Wc* = 0, meaning the neuron completely ignores these channels when computing its response.

**Q: why there are some channels have *Wc* = 0, and what if we want more channels to have *Wc* = 0? how to do that?**

We will visualize the channels with large weights defined by a threshold on `np.abs(Wc)`.


In [ ]:
# only visualize the channels that contribute significantly to the readout
top_channels = np.where(np.abs(Wc) > 0.3)[0]
print(f'top channels: {len(top_channels)} / {len(Wc)}')

We will visualize the top conv2 filters, as in the full model:

In [ ]:
conv2_1x1 = model.core.features.layer1.ds_conv.in_depth_conv.weight.data.cpu().numpy().squeeze()
plot_conv2(conv1_w.squeeze(), conv2_1x1, ichannels=top_channels, Wc=Wc)

**Q: Based on the visualization, what kind of features does the neuron seem to be mainly responding to?**

---

# 8. Preferred Stimuli via Activation Maximization ⏱️ ~30 min  

To better understand neuronal selectivity—and how it relates to the selectivity of individual conv2 channels—we find the input images that **maximize the activation** of either a specific neuron or a specific feature channel.  

These images reveal the visual patterns that individual neurons are most responsive to, and show how the features captured by each conv2 channel contribute to a neuron's selectivity.

First, let's get the responses of each model neuron to the input images:


In [ ]:
# get the model predictions on the train images
neuron_activity_model = model_trainer.test_epoch(model, X_train_t)
neuron_activity_model = neuron_activity_model.squeeze()
print('neuron_activity_model shape:', neuron_activity_model.shape)

################################################################################
# TODO for students: sort the stimuli based on predicted activity
raise NotImplementedError("Student exercise: sort stimuli by model activity")
################################################################################
# Hint:
# - Use np.argsort to rank stimuli by model-predicted activity
# - Reverse the order to show highest activity first
prediction_isort = ...

fig, ax = plt.subplots(1, 1, figsize=(4, 3))
ax.plot(np.arange(len(prediction_isort)), neuron_activity_model[prediction_isort], 'k', label='model')
ax.set_xlabel('stimulus index (sorted by activity)')
ax.set_ylabel('neuron activity')
ax.set_title(f'Neuron {ineuron} activity')

Now, let's compute the activation of each conv2 channel across the input images.  
We apply the spatial readout weights (*Wxy*) to pool each channel's feature map into a single scalar value—summarizing the channel's overall response to each image.


In [ ]:
Nimgs_unique = X_train_t.shape[0]
model.eval()
batch_size = 160
channel_activity = np.zeros((Nimgs_unique, nconv2))
for i in range(0, Nimgs_unique, batch_size):
    images = X_train_t[i:i+batch_size]
    conv2_fv = model.core(images)

    ################################################################################
    # TODO for students: compute the activity of each conv2 channel after spatial pooling
    raise NotImplementedError("Student exercise: apply spatial pooling to conv2 feature maps")
    ################################################################################
    # Hint:
    # - Use torch.einsum to apply Wx and Wy to the spatial dimensions of conv2 features
    # - conv2_fv shape: (batch, channels, height, width)
    # - Wx, Wy shape: (n_neurons, width) and (n_neurons, height)
    # - Result should be (batch, channels)
    wxy_fv = ...
    
    channel_activity[i:i+batch_size] = wxy_fv
print('conv2 channel activity: ', channel_activity.shape)

In [ ]:
# visualize the sorted activity of each channel
channel_activity_valid = channel_activity[:, top_channels]
nchan = len(top_channels)
cmap = plt.get_cmap('plasma', 9)
fig, ax = plt.subplots(1, 1, figsize=(3,3))
for i in range(8):
    ic = i if i<6 else -(8-i)
    sorted_fv = np.sort(channel_activity_valid[:, ic])[::-1]
    ic = i+1 if i<6 else nchan+1-(8-i)
    ax.plot(sorted_fv[:200], color=cmap(i), label=f'channel {ic}')
    if i ==7:
        ax.text(0.5, 0.9-6*0.05, f'channel K', color=cmap(i), transform=ax.transAxes)
    elif i<4:
        ax.text(0.5, 0.9-i*0.05, f'channel {ic}', color=cmap(i), transform=ax.transAxes)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('Stimulus (sorted)')
ax.set_ylabel('Activity')

In [ ]:
# sort the activity of images in each channel
channel_activity_isort = np.argsort(-channel_activity, axis=0)
channel_activity_isort = channel_activity_isort[:, top_channels]
# only take the top 8 images for each channel
channel_activity_isort = channel_activity_isort[:8]

In [ ]:
# generate a mask on the image based on the model Wxy
from minimodel.utils import get_image_mask
Ly, Lx = X_train_t.shape[-2:]
img_mask = get_image_mask(model, Ly=Ly, Lx=Lx)

In [ ]:
valid_wc = Wc[top_channels]
plot_maxstim(valid_wc, X_train_t, prediction_isort, channel_activity_isort, img_mask)


# 9. Wrap-Up & Discussion ⏱️ ~15 min  

In this tutorial, we explored a range of models that predict neural responses in mouse V1, from simple linear regression to biologically inspired convolutional architectures. Along the way, we introduced key concepts such as receptive fields, spatial integration, and feature selectivity—culminating in an interpretable, high-performing model.

### Summary

- **Even simple models can capture structure**:  
  Linear regression and single-layer models with a factorized readout provided a strong baseline, with high interpretability.

- **Factorized readouts offer insight into tuning**:  
  The use of spatial weights ($\mathbf{w}_y$, $\mathbf{w}_x$) and channel weights $\mathbf{w}_c$ enables us independently probe *where* a neuron samples from the visual field and *what* features it prefers.  

- **Two-layer CNNs provide substantially better performance but are less interpretable**:  
  As shown in *Du et al. (2024)*, a shallow model with just two convolutional layers achieves strong performance. However, the neurons are combinations of hundreds of features, making them difficult to interpret.

- **Minimodels achieve equivalent performance with high interpretability**:  
  By training a compact, neuron-specific readout on top of a shared convolutional backbone, the minimodel reproduces the performance of the full model with far fewer parameters—making it both efficient and interpretable.
  We could fit the minimodels with high sparsity in $\mathbf{w}_c$, suggesting that most neurons only combine a smaller set of features.

### Discussion questions

- What are the advantages of shallow models over deeper networks in this context?
- Could this modeling approach generalize to other brain areas or modalities?
- What are the limits of this feedforward model—what kinds of neural dynamics are missing?

### Suggested extensions

If you're interested in pushing this work further, consider exploring:

- How model performance changes with more or fewer conv filters
- Using these models to study invariance using the natural texture stimuli


---
### Acknowledgments

Some of this material was adapted from the Neuromatch Deep Learning Day [tutorials](https://compneuro.neuromatch.io/tutorials/W1D5_DeepLearning/student/W1D5_Tutorial1.html) by Jorge Menendez and Carsen Stringer.